In this notebook I read the original data from MongoDB, and create a new MongoDB with processed data.

* annonymization (replace names with placeholder)
* splitting comments in sentences
* language detection


In [61]:
import pandas as pd
import pymongo

from polyglot.text import Text
from langdetect import detect
from googletrans import Translator
translator = Translator()

import re


In [62]:
conn=pymongo.MongoClient()
db=conn["db_enquestes"]

Annonymization:

To replace teacher names, I am taking all the distinct teacher names from the pdf, splitting the names and looking for those names in the comments.

For every name I add to search list the name with "l'" for example for the name "ana" i search for "ana" and "l'ana".

There are a name that has a word "de", so I am not searching for that word. Also I removed name "mas".

In [63]:
db2 = conn["db_comments_processed"]
global collection
collection = db2.comments_processed

In [64]:
teacher_names = db.enquestes_professors.distinct("professor")
teacher_names = [item.lower().split() for item in teacher_names]
teacher_names = [item for sublist in teacher_names for item in sublist]
teacher_names.append("anna")
teacher_names.append("vitriá")
while 'de' in teacher_names: teacher_names.remove('de')
while 'mas' in teacher_names: teacher_names.remove('mas')
while "nin" in teacher_names: teacher_names.remove("nin")
n = []
for name in teacher_names:
    nn = 'l\''+name
    nc = name+','
    n.append(nn)
    n.append(nc)
teacher_names = teacher_names + n
#teacher_names

In [65]:
for elem in db.enquestes_professors.find():
    pdf_id = elem['_id']
    q1_answers = elem['q1_answers']
    for comment in q1_answers:
        dict = {}
        dict["form_type"] = "enquestes_professors"
        dict["pdf_id"] = pdf_id
        dict["question_nr"] = 1
        language = translator.detect(comment).lang
        if language == 'espt':
            language = 'es'
        dict["language"] = language
        
        ### REPLACE PROFESSOR NAMES WITH "PROFESSOR_NAME"
        c = comment.split()
        c_lower = [x.lower() for x in c]
        for name in teacher_names:
            if name in c_lower:
                ind = c_lower.index(name)
                c[ind] = "PROFESSOR_NAME"
        comment = " ".join(c)

        sents = Text(comment).sentences
        sentences = [str(item) for item in sents]
        dict["sentences"] = sentences
        
        collection.insert_one(dict)
    
    q2_answers = elem['q2_answers']
    for comment in q2_answers:
        dict = {}
        dict["form_type"] = "enquestes_professors"
        dict["pdf_id"] = pdf_id
        dict["question_nr"] = 2
        language = translator.detect(comment).lang
        if language == 'espt':
            language = 'es'
        dict["language"] = language
        
        ### REPLACE PROFESSOR NAMES WITH "PROFESSOR_NAME"
        c = comment.split()
        c_lower = [x.lower() for x in c]
        for name in teacher_names:
            if name in c_lower:
                ind = c_lower.index(name)
                c[ind] = "PROFESSOR_NAME"
        comment = " ".join(c)

        sents = Text(comment).sentences
        sentences = [str(item) for item in sents]
        dict["sentences"] = sentences
        
        collection.insert_one(dict)

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

In [66]:
db2.comments_processed.distinct("language")

['ca', 'es', 'en', 'gl']

In [75]:
docs =  db2.comments_processed.find({"language": "gl"})

In [76]:
for d in docs:
    print(d)

{'_id': ObjectId('5ad9cba52cfaa8fca296aae1'), 'form_type': 'enquestes_professors', 'pdf_id': ObjectId('5ad8db922cfaa8fb4930709a'), 'question_nr': 1, 'language': 'gl', 'sentences': ['Clases amenas e interesantes.']}


In [78]:
docs =  db2.comments_processed.find({"language": "en"})
for d in docs:
    print(d)

{'_id': ObjectId('5ad9cb852cfaa8fca296a90c'), 'form_type': 'enquestes_professors', 'pdf_id': ObjectId('5ad8dab62cfaa8fb49307079'), 'question_nr': 2, 'language': 'en', 'sentences': ['Res']}
{'_id': ObjectId('5ad9cb892cfaa8fca296a946'), 'form_type': 'enquestes_professors', 'pdf_id': ObjectId('5ad8dacc2cfaa8fb4930707c'), 'question_nr': 2, 'language': 'en', 'sentences': ['PROFESSOR_NAME FOR PRESIDENT']}
{'_id': ObjectId('5ad9cb8a2cfaa8fca296a954'), 'form_type': 'enquestes_professors', 'pdf_id': ObjectId('5ad8dad92cfaa8fb4930707e'), 'question_nr': 1, 'language': 'en', 'sentences': ['None']}
{'_id': ObjectId('5ad9cb8a2cfaa8fca296a959'), 'form_type': 'enquestes_professors', 'pdf_id': ObjectId('5ad8dad92cfaa8fb4930707e'), 'question_nr': 2, 'language': 'en', 'sentences': ['One of the worst teachers I have had in the career.']}
{'_id': ObjectId('5ad9cb8a2cfaa8fca296a960'), 'form_type': 'enquestes_professors', 'pdf_id': ObjectId('5ad8dadf2cfaa8fb4930707f'), 'question_nr': 1, 'language': 'en', 'se